In [1]:
%pwd

'f:\\DS\\TextSummarizer\\Text-Summarizer\\research'

In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'f:\\DS\\TextSummarizer\\Text-Summarizer'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_dir: Path
    model_ckpt_file: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [5]:
from textSummarizer.constants import *
from textSummarizer.utils.common_utils import read_yaml, create_directories

In [17]:
class ConfigurationManager:

    def __init__(
            self,
            config_file_path=CONFIG_FILE_PATH,
            params_file_path=PARAMS_FILE_PATH
            ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def fetch_modeltrainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_dir=config.data_dir,
            model_ckpt_file=config.model_ckpt_file,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=params.eval_steps,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps,
        )

        return model_trainer_config


In [8]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

In [26]:
class ModelTrainer:

    def __init__(
            self,
            model_trainer_config: ModelTrainerConfig
            ):
        self.config = model_trainer_config

    def train(self):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt_file)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt_file).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        dataset_samsum_pt = load_from_disk(self.config.data_dir)

        #loading data
        trainer_args = TrainingArguments(
                output_dir=self.config.root_dir, 
                num_train_epochs=self.config.num_train_epochs,
                warmup_steps=self.config.warmup_steps,
                per_device_train_batch_size=self.config.per_device_train_batch_size, 
                per_device_eval_batch_size=self.config.per_device_train_batch_size,
                weight_decay=self.config.weight_decay, 
                logging_steps=self.config.logging_steps,
                evaluation_strategy=self.config.evaluation_strategy, 
                eval_steps=self.config.eval_steps, 
                save_steps=1e6,
                gradient_accumulation_steps=self.config.gradient_accumulation_steps
        )

        trainer = Trainer(
            model=model_pegasus,
            args=trainer_args,
            data_collator=seq2seq_data_collator,
            tokenizer=tokenizer,
            train_dataset=dataset_samsum_pt["test"],
            eval_dataset=dataset_samsum_pt["validation"]
        )

        trainer.train()

        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus samsum model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

In [27]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.fetch_modeltrainer_config()
    model_trainer_config = ModelTrainer(model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-03-28 20:07:30,548: INFO: common_utils: yaml file: config\config.yaml loaded successfully]
[2024-03-28 20:07:30,561: INFO: common_utils: yaml file: params.yaml loaded successfully]
[2024-03-28 20:07:30,565: INFO: common_utils: Created directory at: artifacts]
[2024-03-28 20:07:30,571: INFO: common_utils: Created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Hi\.conda\envs\textS\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
  8%|▊         | 4/51 [23:51<4:19:08, 330.83s/it]